In [3]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer for TensorFlow
model_name = "gpt2"  # Name of the pre-trained model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name)

def generate_text_with_coherence(prompt, max_length=300, memory_cache=None):
    """
    Generates coherent long-form text using prompt engineering and memory.

    Parameters:
        prompt (str): Initial prompt to start the text generation.
        max_length (int): Maximum length of the generated text.
        memory_cache (list): Stores recent generated text chunks for coherence checking.

    Returns:
        str: Generated coherent text.
    """
    # Initialize memory_cache if not provided, to store generated text chunks
    if memory_cache is None:
        memory_cache = []

    # Tokenize input prompt
    input_ids = tokenizer(prompt, return_tensors="tf").input_ids

    # Placeholder for the full output text
    output_text = ""

    # Generate text in chunks to check for coherence
    for _ in range(max_length // 50):  # Loop to generate text in small chunks
        # Generate a text chunk of max 50 new tokens for coherence control
        outputs = model.generate(
            input_ids,
            max_new_tokens=50,  # Generate only new tokens
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated tokens to text
        chunk_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Add the new chunk to memory_cache
        memory_cache.append(chunk_text)

        # Check and ensure coherence of the newly generated chunk
        coherent_text = coherence_check(memory_cache)

        # Append the coherent chunk to the overall output text
        output_text += coherent_text

        # Prepare the next input by tokenizing coherent_text to continue generating
        input_ids = tokenizer(coherent_text, return_tensors="tf").input_ids

        # Maintain memory by keeping only recent chunks (to avoid memory overflow)
        if len(memory_cache) > 5:  # If more than 5 chunks, remove the oldest
            memory_cache.pop(0)

    return output_text

def coherence_check(memory_cache):
    """
    Checks and enforces coherence across the text chunks in memory.

    Parameters:
        memory_cache (list): Recent generated chunks of text.

    Returns:
        str: Coherent text adjusted based on recent memory.
    """
    # Check for thematic consistency by comparing with the previous chunk
    if len(memory_cache) > 1:
        # Extract first few words of the recent and previous chunks
        recent_chunk = memory_cache[-1].split()[:10]
        prev_chunk = memory_cache[-2].split()[:10]

        # Ensure thematic alignment by checking for common words or ideas
        if set(recent_chunk).intersection(set(prev_chunk)):
            return memory_cache[-1]  # Return chunk if it aligns thematically

    # If no thematic overlap, add a connective phrase to maintain flow
    connectives = ["therefore", "however", "in addition", "consequently"]
    coherent_text = f"{connectives[len(memory_cache) % len(connectives)]} {memory_cache[-1]}"

    return coherent_text

# Define the initial prompt for text generation
initial_prompt = "Discuss the impact of climate change on polar bear habitats."

# Generate coherent long-form text with the specified prompt and settings
generated_text = generate_text_with_coherence(initial_prompt, max_length=300)
print("Generated Coherent Text:\n", generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Generated Coherent Text:
 however Discuss the impact of climate change on polar bear habitats.

"The polar bear population is declining, and the population of the polar bear is declining," said Dr. David S. Siegel, a professor of ecology and evolutionary biology at the University of California, Berkeley. "The polar bear population ishowever Discuss the impact of climate change on polar bear habitats.

"The polar bear population is declining, and the population of the polar bear is declining," said Dr. David S. Siegel, a professor of ecology and evolutionary biology at the University of California, Berkeley. "The polar bear population is declining, and the population of the polar bear is declining."

The study, published in the journal PLOS ONE, found that the polar bear population has declined by about 10 percent since the mid-1990s.

"The polar bearhowever Discuss the impact of climate change on polar bear habitats.

"The polar bear population is declining, and the population of the p